# Fine-tuning de Llama 3.2 1B avec LoRA pour la tâche NLI

**Luan DECHERY – Mathis DERENNE – Enrique NASSEH FREITAS**

M2 MALIA & MIASH – Université Lumière Lyon 2

Janvier 2026

---

## 1. Encodeur vs Décodeur

L'architecture Transformer repose sur un mécanisme d'attention qui calcule les relations entre tous les mots d'une séquence :

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

où $Q$, $K$ et $V$ sont des projections linéaires de l'entrée. La différence entre encodeur et décodeur réside dans les positions que chaque mot peut observer.

L'**encodeur** utilise une attention bidirectionnelle : chaque mot accède à tous les autres mots de la séquence. Cette architecture est adaptée aux tâches de compréhension (classification, NLI). Le pré-entraînement s'effectue par *Masked Language Modeling* où le modèle prédit des mots masqués. BERT et CamemBERT illustrent cette approche.

Le **décodeur** utilise une attention causale : chaque mot ne voit que les mots précédents. Cette contrainte simule la génération de texte. Le pré-entraînement repose sur la prédiction du mot suivant. GPT et Llama adoptent cette architecture, mais peuvent être adaptés à la classification via des prompts.

| | **Encodeur** | **Décodeur** |
|---|---|---|
| Attention | Bidirectionnelle | Causale |
| Pré-entraînement | Masked LM | Next Token Prediction |
| Tâches | Classification, NLI | Génération, Chat |
| Exemples | BERT, CamemBERT | GPT, Llama |

---

## 2. Llama 3.2 : Modèle 1B vs 3B

Llama 3.2 est une famille de modèles open-source développée par Meta (septembre 2024). Elle intègre plusieurs améliorations :
- **RMSNorm** : normalisation stable
- **RoPE** : encodage positionnel relatif
- **GQA** : partage Key/Value entre têtes
- **SwiGLU** : activation améliorée

| | **1B** | **3B** |
|---|---|---|
| Paramètres | 1.24 milliard | 3.21 milliards |
| Couches | 16 | 28 |
| Dimension cachée | 2048 | 3072 |
| Mémoire GPU (float16) | ~2.5 Go | ~6.5 Go |

Nous avons choisi le modèle **1B** car il permet un entraînement sur GPU Kaggle T4 (16 Go) tout en offrant des capacités suffisantes pour une classification à 3 classes. Le modèle 3B nécessiterait plus de ressources et de temps.

---

## 3. Méthode LoRA

Le fine-tuning classique modifie tous les paramètres du modèle, ce qui pose des problèmes de mémoire et de temps pour un modèle d'1 milliard de paramètres. **LoRA** (Low-Rank Adaptation) résout ce problème en ajoutant une modification de rang faible :

$$h = Wx + BAx$$

où $W$ est la matrice originale (gelée), $B \in \mathbb{R}^{d \times r}$ et $A \in \mathbb{R}^{r \times d}$ avec $r \ll d$. Seules $A$ et $B$ sont entraînées.

Pour une matrice $4096 \times 4096$ avec $r = 16$ : le fine-tuning classique entraîne 16.7M paramètres, LoRA n'en entraîne que 131K, soit une **réduction de 99.2%**.

| **Paramètre** | **Valeur** | **Rôle** |
|---|---|---|
| $r$ | 16 | Rang des matrices A et B |
| $\alpha$ | 32 | Facteur de mise à l'échelle |
| Modules | q, k, v, o_proj | Projections d'attention |
| Dropout | 0.05 | Régularisation |

---

## 4. Résultats expérimentaux

### 4.1 Données et configuration

Le corpus contient des paires de phrases en français : 5 010 exemples d'entraînement et 2 490 de test, équilibrés entre les trois classes (entailment, contradiction, neutral).

| **Paramètre** | **Valeur** |
|---|---|
| Modèle | Llama 3.2 1B |
| Paramètres entraînables | 0.52% |
| Epochs | 2 |
| Batch size | 16 |
| Learning rate | $1 \times 10^{-4}$ |

### 4.2 Performances

| **Classe** | **Precision** | **Recall** | **F1-score** | **Support** |
|---|---|---|---|---|
| Contradiction | 0.7902 | 0.7217 | 0.7544 | 830 |
| Entailment | 0.7198 | 0.7675 | 0.7429 | 830 |
| Neutral | 0.6222 | 0.6349 | 0.6285 | 830 |
| **Accuracy** | – | – | **0.7080** | 2490 |
| Macro avg | 0.7107 | 0.7080 | 0.7086 | 2490 |

Le modèle atteint une accuracy de **70.80%**, bien au-dessus de la baseline aléatoire (33.33%). Les classes *contradiction* et *entailment* obtiennent des F1-scores similaires (~0.75), tandis que la classe *neutral* présente des performances inférieures (F1 = 0.63), ce qui s'explique par sa nature plus ambiguë. En n'entraînant que 0.52% des paramètres, LoRA permet d'adapter efficacement un décodeur à une tâche de classification.

---

## 5. Conclusion

Ce travail montre que les décodeurs autorégressifs peuvent être adaptés à des tâches de classification via des prompts, et que LoRA permet un fine-tuning efficace avec moins de 1% des paramètres. Cette approche rend accessible l'adaptation de grands modèles sur du matériel grand public.